## General setup

To make sure things are working and `hepdata_lib` is available, run the following command:

In [45]:
import hepdata_lib

## Creating your HEPData submission

The `Submission` object represents the whole HEPData entry and thus carries the top-level meta data that is equally valid for all the tables and variables you may want to enter. The object is also used to create the physical submission files you will upload to the HEPData web interface.

When using `hepdata_lib` to make an entry, you always need to create a `Submission` object. Let's do that now, and then add data to it step by step:

In [46]:
from hepdata_lib import Submission
submission = Submission()

In general, a `Submission` should contain details on the actual analysis such as it's abstract as well as links to the actual publication. The abstract should be in a plain text file. For `inspire` there's a special `record_id`, while for links to `arXiv` etc. one should use plain hyperlinks.

In [47]:
submission.read_abstract("abstract.txt")
#submission.add_link("Webpage with all figures and tables", "https://cms-results.web.cern.ch/cms-results/public-results/publications/B2G-16-029/")
#submission.add_link("arXiv", "http://arxiv.org/abs/arXiv:1802.09407")
#submission.add_record_id(1657397, "inspire")

Adding CalcHEP model and LHE headers

In [48]:
submission.add_additional_resource("CalcHEP model.","HNmodel.tar", copy_file=True)
submission.add_additional_resource("CalcHEP LHE headers.","LHEheaders.tar", copy_file=True)

## Adding a table/figure

In HEPData, figures and table will both be `Table` objects. The example here shows reading a plain text file containing the signal effiency times acceptance as a function of resonance mass for different signal models. The file has been uploaded to the `example_files` directory. For your submission, create a new directory, e.g. using the analysis identifier.

Let's have a look at the file:

In [49]:
!head cutflowM500.txt

SELECTION           mumujj   eejj
---------           ------  -----
Trigger             0.692   0.515
LeadingLepton>150GeV     0.692  0.515
SubleadingLepton>100GeV   0.692  0.515  
m(ll)>300GeV              0.659  0.491
numberOfFatJets>0         0.379  0.284      


The first column is the mass value, the other columns contain the efficiency times acceptance values.

Let's create the table/figure. First, we need to give it a name, which is usually just the identifier in the paper, here "Figure 1". The table also needs a description, which is usually the caption. You also need to describe the location, i.e. where to find it in the publication:

In [50]:
from hepdata_lib import Table
table = Table("Cut-flow table")
table.description = "Cut-flow table mN=0.5TeV, electron, muon channel, 2016."
table.location = "Additional material"

Now we need to provide more information on what is actually shown, which is done via `keywords`. The ones that are available can be taken from the documentation:
- [Observables](https://hepdata-submission.readthedocs.io/en/latest/keywords/observables.html)
- [Phrases](https://hepdata-submission.readthedocs.io/en/latest/keywords/phrases.html)
- [Particles](https://hepdata-submission.readthedocs.io/en/latest/keywords/partlist.html)

In [51]:
table.keywords["observables"] = ["ACC", "EFF"]

Let's read in the file. For this purpose, `numpy` is very handy. Since the first two rows are the header, we skip them:

In [52]:
import numpy as np
data = np.loadtxt("cutflowM500.txt", skiprows=2, usecols=range(1,3))

In [53]:
dataTXT = np.loadtxt("cutflowM500.txt", skiprows=2, usecols=range(0), dtype="str")

`numpy` stores the content as arrays. You can actually see that the entry that was labelled as `NaN` is correctly read in:

In [54]:
from __future__ import print_function
print(data)

[[0.692 0.515]
 [0.692 0.515]
 [0.692 0.515]
 [0.659 0.491]
 [0.379 0.284]]


We will now use this for our `Variable` definitions. The x-axis is usually the independent variable (`is_independent=True`), whereas the other ones are dependent (i.e. a function of the former). You also need to declare whether the variable is binned or not as well as the units. Similar as for the `keywords` used above, it is again important to provide additional information that can be found via the HEPData web interface using the observables and particles linked above. The values assigned are just slices of the `data` array:

In [55]:
from hepdata_lib import Variable
d = Variable("Selection", is_independent=True, is_binned=False, units="")
d.values = dataTXT[:,0]

Effmumujj = Variable("Efficiency mumujj", is_independent=False, is_binned=False, units="")
Effmumujj.values = data[:,0]
Effmumujj.add_qualifier("SQRT(S)", 13, "TeV")

Effeejj = Variable("Efficiency eejj", is_independent=False, is_binned=False, units="")
Effeejj.values = data[:,1]
Effeejj.add_qualifier("SQRT(S)", 13, "TeV")

table.add_variable(d)
table.add_variable(Effmumujj)
table.add_variable(Effeejj)

This is all that's needed for the table/figure. We still need it to the submission:

In [56]:
submission.add_table(table)

Once you've added all tables/figures and the general submission details, you should add a few more keywords to all tables for better identification and searchability, e.g. the centre-of-mass energy:

In [57]:
for table in submission.tables:
    table.keywords["cmenergies"] = [13000]

# Reading histograms for SR plots

### Electron channel

In [58]:
from hepdata_lib import Table
table = Table("Figure 4a")
table.description = "Distributions of \mllj for the data, and the post-fit backgrounds (stacked histograms), in the SRs of the \eeqq channel. The template for one signal hypothesis is shown overlaid as a yellow solid line. The overflow is included in the last bin. The middle panels show ratios of the data to the pre-fit background prediction and post-fit background yield as red open squares and blue points, respectively. The gray band in the middle panels indicates the systematic component of the post-fit uncertainty. The lower panels show the distributions of the pulls, defined in the text."
table.location = "Data from Figure 4 (upper left)."
table.keywords["observables"] = ["N"]
table.add_image("Figure_004-a.pdf")

In [59]:
from hepdata_lib import RootFileReader

reader = RootFileReader("eejj_PostFit_histograms_L13_M05_PLOT_SR.root")
reader_data = RootFileReader("eejj_PostFit_histograms_L13_M05_PLOT_SR.root")
reader_signal = RootFileReader("eejj_PostFit_histograms_L13_M05_PLOT_SR.root")

TotalBackground = reader.read_hist_1d("SR_postfit/TotalBkg")
#TT = reader.read_hist_1d("shapes_prefit/cat0_singleH/TT")
#QCD = reader.read_hist_1d("shapes_prefit/cat0_singleH/QCDTT")
#WJets = reader.read_hist_1d("shapes_prefit/cat0_singleH/WJets")
#ZJets = reader.read_hist_1d("shapes_prefit/cat0_singleH/ZJets")

Data = reader_data.read_hist_1d("SR_prefit/data_obs")

signal = reader_signal.read_hist_1d("SR_postfit/TotalSig")

In [60]:
Data['x']= Data['x'][2:]
Data['x_edges']= Data['x_edges'][2:]
Data['x_labels']= Data['x_labels'][2:]
Data['y']= Data['y'][2:]
Data['dy']= Data['dy'][2:]

In [61]:
#Data['y'][6]=''
#Data['dy'][6]=''
Data

{'x': [0.5, 0.7, 0.9, 1.2, 1.7, 2.75, 6.75],
 'y': [17.0, 316.0, 442.0, 529.0, 201.0, 55.0, 3.0],
 'x_edges': [(0.4, 0.6),
  (0.5999999999999999, 0.8),
  (0.8, 1.0),
  (1.0, 1.4),
  (1.4, 2.0),
  (2.0, 3.5),
  (3.5, 10.0)],
 'x_labels': ['', '', '', '', '', '', ''],
 'dy': [4.123105625617661,
  17.776388834631177,
  21.02379604162864,
  23.0,
  14.177446878757825,
  7.416198487095663,
  1.7320508075688772]}

In [62]:
TotalBackground

{'x': [0.1, 0.30000000000000004, 0.5, 0.7, 0.9, 1.2, 1.7, 2.75, 6.75],
 'y': [9.442180370911046e-09,
  0.03634917363524437,
  21.11818504333496,
  311.08721923828125,
  428.8046875,
  478.2311096191406,
  200.43923950195312,
  56.502838134765625,
  1.5858254432678223],
 'x_edges': [(0.0, 0.2),
  (0.20000000000000004, 0.4),
  (0.4, 0.6),
  (0.5999999999999999, 0.8),
  (0.8, 1.0),
  (1.0, 1.4),
  (1.4, 2.0),
  (2.0, 3.5),
  (3.5, 10.0)],
 'x_labels': ['', '', '', '', '', '', '', '', ''],
 'dy': [2.163529638259804e-10,
  0.0293767535903556,
  2.3873863536961752,
  9.919429449179514,
  10.031811312339505,
  14.671918175227677,
  8.19926291544975,
  4.182018918754771,
  0.4180984010163905]}

In [63]:
TotalBackground['x']= TotalBackground['x'][2:]
TotalBackground['x_edges']= TotalBackground['x_edges'][2:]
TotalBackground['x_labels']= TotalBackground['x_labels'][2:]
TotalBackground['y']= TotalBackground['y'][2:]
TotalBackground['dy']= TotalBackground['dy'][2:]

In [64]:
signal['x']= signal['x'][2:]
signal['x_edges']= signal['x_edges'][2:]
signal['x_labels']= signal['x_labels'][2:]
signal['y']= signal['y'][2:]
signal['dy']= signal['dy'][2:]

In [65]:
signal

{'x': [0.5, 0.7, 0.9, 1.2, 1.7, 2.75, 6.75],
 'y': [0.005832682363688946,
  2.7306528091430664,
  13.532551765441895,
  45.658103942871094,
  67.69922637939453,
  69.75869750976562,
  10.573692321777344],
 'x_edges': [(0.4, 0.6),
  (0.5999999999999999, 0.8),
  (0.8, 1.0),
  (1.0, 1.4),
  (1.4, 2.0),
  (2.0, 3.5),
  (3.5, 10.0)],
 'x_labels': ['', '', '', '', '', '', ''],
 'dy': [0.0018763037115665676,
  0.07963309571361413,
  0.2656156441632101,
  0.9410435059855214,
  1.043453013932441,
  1.4188860176129992,
  0.4829481183501832]}

In [66]:
from hepdata_lib import Variable, Uncertainty

# x-axis: B quark mass
mmed = Variable("$m(eeJ)$", is_independent=True, is_binned=True, units="TeV")
mmed.values = [ (0.4,0.6), (0.6,0.8), (0.8,1.0), (1.0,1.4), (1.4,2.0), (2.0,3.5), (3.5,10.0)]
#mmed.values = signal["x"]

# y-axis: N events
sig = Variable("Number of signal events", is_independent=False, is_binned=False, units="")
sig.values = signal["y"]

totalbackground = Variable("Number of background events", is_independent=False, is_binned=False, units="")
totalbackground.values = TotalBackground["y"]

#tt = Variable("Number of ttbar events", is_independent=False, is_binned=False, units="")
#tt.values = TT["y"]

#qcd = Variable("Number of qcd events", is_independent=False, is_binned=False, units="")
#qcd.values = QCD["y"]

#wjets = Variable("Number of wjets events", is_independent=False, is_binned=False, units="")
#wjets.values = WJets["y"]

#zjets = Variable("Number of zjets events", is_independent=False, is_binned=False, units="")
#zjets.values = ZJets["y"]

data = Variable("Number of data events", is_independent=False, is_binned=False, units="")
data.values = Data["y"]

In [67]:
from hepdata_lib import Uncertainty

unc_totalbackground = Uncertainty("total uncertainty", is_symmetric=True)
unc_totalbackground.values = TotalBackground["dy"]

unc_data = Uncertainty("Poisson errors", is_symmetric=True)
unc_data.values = Data["dy"]


totalbackground.add_uncertainty(unc_totalbackground)
data.add_uncertainty(unc_data)

In [68]:
table.add_variable(mmed)
#table.add_variable(sig)
table.add_variable(totalbackground)
#table.add_variable(tt)
#table.add_variable(qcd)
#table.add_variable(zjets)
#table.add_variable(wjets)
table.add_variable(data)

submission.add_table(table)

### Muon channel

In [69]:
from hepdata_lib import Table
table = Table("Figure 4b")
table.description = "Distributions of \mllj for the data, and the post-fit backgrounds (stacked histograms), in the SRs of the \mmqq channel. The template for one signal hypothesis is shown overlaid as a yellow solid line. The overflow is included in the last bin. The middle panels show ratios of the data to the pre-fit background prediction and post-fit background yield as red open squares and blue points, respectively. The gray band in the middle panels indicates the systematic component of the post-fit uncertainty. The lower panels show the distributions of the pulls, defined in the text."
table.location = "Data from Figure 4 (upper right)."
table.keywords["observables"] = ["N"]
table.add_image("Figure_004-b.pdf")


from hepdata_lib import RootFileReader
reader = RootFileReader("mumujj_PostFit_histograms_L13_M05_PLOT_SR.root")
reader_data = RootFileReader("mumujj_PostFit_histograms_L13_M05_PLOT_SR.root")
reader_signal = RootFileReader("mumujj_PostFit_histograms_L13_M05_PLOT_SR.root")
TotalBackground = reader.read_hist_1d("SR_postfit/TotalBkg")
Data = reader_data.read_hist_1d("SR_prefit/data_obs")
signal = reader_signal.read_hist_1d("SR_postfit/TotalSig")

Data['x']= Data['x'][2:]
Data['x_edges']= Data['x_edges'][2:]
Data['x_labels']= Data['x_labels'][2:]
Data['y']= Data['y'][2:]
Data['dy']= Data['dy'][2:]

TotalBackground['x']= TotalBackground['x'][2:]
TotalBackground['x_edges']= TotalBackground['x_edges'][2:]
TotalBackground['x_labels']= TotalBackground['x_labels'][2:]
TotalBackground['y']= TotalBackground['y'][2:]
TotalBackground['dy']= TotalBackground['dy'][2:]

signal['x']= signal['x'][2:]
signal['x_edges']= signal['x_edges'][2:]
signal['x_labels']= signal['x_labels'][2:]
signal['y']= signal['y'][2:]
signal['dy']= signal['dy'][2:]


from hepdata_lib import Variable, Uncertainty

mmed = Variable("$m(\mu \mu J)$", is_independent=True, is_binned=True, units="TeV")
#mmed.values = signal["x"]
mmed.values = [ (0.4,0.6), (0.6,0.8), (0.8,1.0), (1.0,1.4), (1.4,2.0), (2.0,3.5), (3.5,10.0)]

sig = Variable("Number of signal events", is_independent=False, is_binned=False, units="")
sig.values = signal["y"]
totalbackground = Variable("Number of background events", is_independent=False, is_binned=False, units="")
totalbackground.values = TotalBackground["y"]
data = Variable("Number of data events", is_independent=False, is_binned=False, units="")
data.values = Data["y"]


from hepdata_lib import Uncertainty
unc_totalbackground = Uncertainty("total uncertainty", is_symmetric=True)
unc_totalbackground.values = TotalBackground["dy"]
unc_data = Uncertainty("Poisson errors", is_symmetric=True)
unc_data.values = Data["dy"]
totalbackground.add_uncertainty(unc_totalbackground)
data.add_uncertainty(unc_data)


table.add_variable(mmed)
#table.add_variable(sig)
table.add_variable(totalbackground)
table.add_variable(data)
submission.add_table(table)

In [70]:
signal

{'x': [0.5, 0.7, 0.9, 1.2, 1.7, 2.75, 6.75],
 'y': [0.004714559763669968,
  3.914280891418457,
  17.784940719604492,
  62.210391998291016,
  91.4008560180664,
  88.45048522949219,
  14.164371490478516],
 'x_edges': [(0.4, 0.6),
  (0.5999999999999999, 0.8),
  (0.8, 1.0),
  (1.0, 1.4),
  (1.4, 2.0),
  (2.0, 3.5),
  (3.5, 10.0)],
 'x_labels': ['', '', '', '', '', '', ''],
 'dy': [0.0006727118442834681,
  0.08976382381012545,
  0.29699275886028886,
  0.9697476264389274,
  2.0450207885489737,
  2.4206717712988324,
  0.9139852474445072]}

# Figure 5 and table with theoretical cross-sections

In [71]:
from hepdata_lib import Table
table = Table("Figure 5a")
table.description = "Expected (black dashed lines with green dark and yellow light bands) and observed (solid blue lines) limits on the product of cross section and branching fraction for the electron channel.  The uncertainty bands account for post-fit statistical and systematic uncertainty. The magenta dot-dashed lines denote the model cross sections for the benchmark scale parameter $\Lambda=13 TeV$."
table.location = "Figure 5 (electron channel)."
table.keywords["observables"] = ["N"]
table.add_image("Figure_005-a.pdf")
submission.add_table(table)

In [72]:
from hepdata_lib import Table
table = Table("Figure 5b")
table.description = "Expected (black dashed lines with green dark and yellow light bands) and observed (solid blue lines) limits on the product of cross section and branching fraction for the muon channel.  The uncertainty bands account for post-fit statistical and systematic uncertainty. The magenta dot-dashed lines denote the model cross sections for the benchmark scale parameter $\Lambda=13 TeV$."
table.location = "Figure 5 (muon channel)."
table.keywords["observables"] = ["N"]
table.add_image("Figure_005-b.pdf")
submission.add_table(table)

In [73]:
from hepdata_lib import Table
table = Table("Theoretical cross-sections table")
table.description = "Theoretical cross-sections for heavy composite majorana neutrino signals for $\Lambda$=13 TeV."
table.location = "Additional material for Figures 5 a,b"

table.keywords["observables"] = ["ACC", "EFF"]


import numpy as np
data = np.loadtxt("xsec.txt", skiprows=2)


In [74]:
from __future__ import print_function
print(data)

[[5.000e-01 5.716e-03]
 [1.000e+00 2.819e-03]
 [2.000e+00 8.207e-04]
 [5.000e+00 1.425e-05]
 [8.000e+00 9.135e-08]]


In [75]:
from hepdata_lib import Variable
mass = Variable("Mass (TeV)", is_independent=True, is_binned=False, units="")
mass.values = data[:,0]

xsec = Variable("cross-section for $\Lambda$=13 TeV (pb)", is_independent=False, is_binned=False, units="")
xsec.values = data[:,1]

table.add_variable(mass)
table.add_variable(xsec)

submission.add_table(table)

for table in submission.tables:
    table.keywords["cmenergies"] = [13000]

# Figure 6

In [76]:
from hepdata_lib import Table
table = Table("Figure 6a")
table.description = "Expected (black dashed lines with green dark and yellow light bands) and observed (solid blue lines) limits in the $(m_{N},\Lambda)$ plane of the composite model for the electron channel. The gray shading indicates the region where $m_{N}$ would exceed $\Lambda$, the EFT scale parameter, and the three solid magenta lines in the lower part of the plots represent the fraction of the signal-model phase space that satisfies the unitarity condition in the EFT approximation."
table.location = "Figure 6 (electron channel)."
table.keywords["observables"] = ["N"]
table.add_image("Figure_006-a.pdf")
submission.add_table(table)

In [77]:
from hepdata_lib import Table
table = Table("Figure 6b")
table.description = "Expected (black dashed lines with green dark and yellow light bands) and observed (solid blue lines) limits in the $(m_{N},\Lambda)$ plane of the composite model for the muon channel. The gray shading indicates the region where $m_{N}$ would exceed $\Lambda$, the EFT scale parameter, and the three solid magenta lines in the lower part of the plots represent the fraction of the signal-model phase space that satisfies the unitarity condition in the EFT approximation."
table.location = "Figure 6 (muon channel)."
table.keywords["observables"] = ["N"]
table.add_image("Figure_006-b.pdf")
submission.add_table(table)

## Output file

Now it's time to create the submission for the upload. Here, we choose `example_output` as output directory:

In [78]:
outdir = "HN_output"
submission.create_files(outdir,remove_old=True)

In the working directory, you will now find a `submission.tar.gz` file, which you can use for uploading to your HEPData sandbox:

In [79]:
!ls submission.tar.gz

submission.tar.gz


And the `example_output` directory will contain the generated `yaml` files:

In [80]:
!ls HN_output

Figure_004-a.png  cut-flow_table.yaml  theoretical_cross-sections_table.yaml
Figure_004-b.png  figure_4a.yaml       thumb_Figure_004-a.png
Figure_005-a.png  figure_4b.yaml       thumb_Figure_004-b.png
Figure_005-b.png  figure_5a.yaml       thumb_Figure_005-a.png
Figure_006-a.png  figure_5b.yaml       thumb_Figure_005-b.png
Figure_006-b.png  figure_6a.yaml       thumb_Figure_006-a.png
HNmodel.tar	  figure_6b.yaml       thumb_Figure_006-b.png
LHEheaders.tar	  submission.yaml
